In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



In [3]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

In [4]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

# Converting function to UDF 
convertUDF = udf(lambda z: convertCase(z),StringType())

# Converting function to UDF 
# StringType() is by default hence not required 
"""
convertUDF = udf(lambda z: convertCase(z)) 
"""

'\nconvertUDF = udf(lambda z: convertCase(z)) \n'

In [5]:
df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
   .show(truncate=False)

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+



In [6]:
def upperCase(str):
    return str.upper()

In [7]:
upperCaseUDF = udf(lambda z:upperCase(z),StringType())   

df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
  .show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



In [8]:
""" Using UDF on SQL """

spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+



In [9]:
"""Using UDF with Annotation"""

@udf(returnType=StringType()) 
def upperCase(str):
    return str.upper()

df.withColumn("Cureated Name", upperCase(col("Name"))) \
.show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



In [12]:
"""
PySpark/Spark에서 하위 표현식의 평가 순서를 보장하지 않음,
표현식이 왼쪽 -> 오른쪽으로 평가되거나 다른 고정된 순서로
평가되는 것을 보장하지 않음. PySpark는 쿼리 최적화 및 계획을 위해 실행을 재정렬하므로
AND, OR, WHERE 및 HAVING  표현식에 부작용이 있음
"""

'\nPySpark/Spark에서 하위 표현식의 평가 순서를 보장하지 않음,\n표현식이 왼쪽 -> 오른쪽으로 평가되거나 다른 고정된 순서로\n평가되는 것을 보장하지 않음. PySpark는 쿼리 최적화 및 계획을 위해 실행을 재정렬하므로\nAND, OR, WHERE 및 HAVING  표현식에 부작용이 있음\n'

In [13]:
# NULL 검사 처리
""" null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")

spark.sql("select convertUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |null        |
+-----+------------+



PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\qkrwn\AppData\Local\Temp\ipykernel_10192\2396831287.py", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'


In [14]:
"""
외부에서 null을 확인하는 대신 UDF 함수 내부에서 null을 확인하는 것이 항상 가장 좋습니다.
어쨌든 UDF에서 null 검사를 할 수 없으면,
IF 또는 CASE WHEN을 사용하여 null을 검사하고 UDF를 조건부로 호출하십시오.
"""

'\n외부에서 null을 확인하는 대신 UDF 함수 내부에서 null을 확인하는 것이 항상 가장 좋습니다.\n어쨌든 UDF에서 null 검사를 할 수 없으면,\nIF 또는 CASE WHEN을 사용하여 null을 검사하고 UDF를 조건부로 호출하십시오.\n'

In [15]:
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False) 

+------------------+
|_nullsafeUDF(Name)|
+------------------+
|John Jones        |
|Tracey Smith      |
|Amy Sanders       |
|                  |
+------------------+

+-----+-----------+
|Seqno|Name       |
+-----+-----------+
|1    |John Jones |
+-----+-----------+

